In [ ]:
import numpy as np
from dataclasses import dataclass

# ==========================================
# 1. Define Product Anatomy (Ref: Section 2.1)
# ==========================================

@dataclass
class MARC_Instrument:
    """
    Defines the structural parameters of the Multi-Asset Autocallable.
    """
    # Asset parameters
    tickers: list[str]          # List of N underlying assets
    initial_spots: np.array     # S_i(0)
    
    # Barrier Logic (Section 2.1.2 & 2.1.3)
    barrier_autocall: float     # B_KO (e.g., 1.00 or 100%)
    barrier_knock_in: float     # B_KI (e.g., 0.60 or 60%)
    is_american_ki: bool        # True = Continuous observation, False = European
    
    # Coupon Logic (Section 2.1.4)
    coupon_rate: float
    barrier_coupon: float       # Level to trigger coupon payment
    with_memory: bool           # Phoenix/Memory feature toggle
    
    # Dates
    observation_dates: np.array # Array of discrete observation times {t_1...t_M}
    maturity: float             # T

# ==========================================
# 2. The Correlation Engine (Ref: Section 2.2.1)
# ==========================================

class CorrelationEngine:
    """
    Handles the mathematical stability of the correlation matrix.
    Critically, this fixes 'broken' matrices from dirty market data.
    """
    
    def __init__(self, raw_correlation_matrix):
        self.raw_matrix = raw_correlation_matrix
        self.N = raw_correlation_matrix.shape[0]

    def get_stable_cholesky(self):
        """
        Returns the Lower Triangular Matrix L such that L * L.T = Sigma.
        Includes error handling for non-PSD matrices.
        """
        try:
            # Attempt standard Cholesky first
            return np.linalg.cholesky(self.raw_matrix)
        except np.linalg.LinAlgError:
            # TRIGGER: Section 2.2.1 "Technical Constraint - Positive Semi-Definiteness"
            # If raw Cholesky fails, the matrix is not PSD.
            # We must project it to the nearest valid matrix.
            print("WARNING: Non-PSD Matrix detected. Applying Spectral Correction...")
            clean_matrix = self._spectral_projection(self.raw_matrix)
            return np.linalg.cholesky(clean_matrix)

    def _spectral_projection(self, matrix):
        """
        Reconstructs the matrix using only positive eigenvalues.
        This is a simplified version of Higham's Algorithm.
        """
        # 1. Eigen Decomposition
        eigenvals, eigenvecs = np.linalg.eigh(matrix)
        
        # 2. Floor negative eigenvalues to a small epsilon (or 0)
        eigenvals = np.maximum(eigenvals, 1e-8)
        
        # 3. Reconstruct Matrix
        # Sigma_new = V * Lambda_clipped * V^T
        T = 1 / (eigenvals**0.5)
        # (Mathematical reconstruction logic omitted for brevity, 
        # but ensures the diagonal remains 1.0)
        
        reconstructed = (eigenvecs * eigenvals) @ eigenvecs.T
        
        # 4. Enforce Correlation property: Diagonal must be exactly 1.0
        # Normalize to ensure unit diagonal
        d = np.sqrt(np.diag(reconstructed))
        reconstructed = reconstructed / np.outer(d, d)
        
        return reconstructed

# ==========================================
# Execution Example
# ==========================================

# 1. Setup the Instrument
marc_note = MARC_Instrument(
    tickers=["SPX", "SX5E", "NKY"],
    initial_spots=np.array([4200.0, 4100.0, 32000.0]),
    barrier_autocall=1.0,
    barrier_knock_in=0.6,
    is_american_ki=True,       # Requires Brownian Bridge in step 4
    coupon_rate=0.08,
    barrier_coupon=0.7,
    with_memory=True,
    observation_dates=np.array([1.0, 2.0, 3.0]),
    maturity=3.0
)

# 2. Prepare the Math Kernel 
# Assume we have a slightly "broken" correlation matrix from data
dirty_corr = np.array([
    [1.0, 0.9, 0.9],
    [0.9, 1.0, 0.9],
    [0.9, 0.9, 1.0] 
]) 
# (Note: Valid correlations sometimes fail Cholesky due to floating point noise)

engine = CorrelationEngine(dirty_corr)
L_matrix = engine.get_stable_cholesky()

print("Initialization Complete. Cholesky Factor L is ready for Monte Carlo.")